In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']

import numpy as np 
import pandas as pd 
import tensorflow as tf
import time
import math

from datetime import datetime
import os
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8,7)
%matplotlib inline

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#Data parameters
num_inputs = 28
num_classes = 10
num_steps=28

old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.examples.tutorials.mnist import input_data

#Data parameters
num_inputs = 28
num_classes = 10
num_steps=28

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
Xtrain = mnist.train.images  # Returns np.array
Xtrain = Xtrain.reshape((-1, num_steps, num_inputs))
ytrain = np.asarray(mnist.train.labels, dtype=np.int32)
Xtest = mnist.test.images  # Returns np.array
Xtest = Xtest.reshape([-1, num_steps, num_inputs])
ytest = np.asarray(mnist.test.labels, dtype=np.int32)

tf.logging.set_verbosity(old_v)

print(Xtrain.shape)
print(ytrain.shape)
print(Xtest.shape)
print(ytest.shape)

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 28, 28)
(55000, 10)
(10000, 28, 28)
(10000, 10)


In [3]:
def get_state_variables(batch_size, cell):
    """Returns TF Variables for storing the cell and hidden states of an LSTM.
    A stateful LSTM keeps its hidden and cell states after a forward propagation run and uses the
    stored states in the next forward propagation run independently of the number of steps the
    LSTM is unrolled for. For implementing such an LSTM in TF, you need to store the state in
    variables. This function returns the state variables of an LSTM cell.
    A comprehensive case for building a multilayer LSTM with a dynamic batch size and number of time
    steps is given below:
    inputs_one_hot = tf.one_hot(inputs, self.vocab_size)
    # inputs_one_hot will have shape (batch_size, num_timesteps, vocab_size)
    batch_size, num_timesteps = tf.shape(inputs_one_hot)[0], tf.shape(inputs_one_hot)[1]
    # For each layer, get the initial state. state will be a tuple of LSTMStateTuples. Get
    # the variables in their own scope so that, we can exclude them from being saved.
    with tf.variable_scope("lstm_state"):
        state = get_state_variables(cell, max_batch_size)
    # Unroll the LSTM
    initial_state = get_state_variables_for_batch(state, batch_size)
    rnn_outputs, new_state = tf.nn.dynamic_rnn(self._cell, inputs_one_hot,
                                               initial_state=initial_state)
    # Only get the outputs of the used batches
    rnn_outputs = rnn_outputs[:batch_size]
    # Add an operation to update the states with the last state tensors
    self._update_state_op = get_state_update_op(state, new_state)
    Args:
        cell (tf.contrib.rnn.MultiRNNCell): An MultiRNNCell consisting of multiple LSTMCells.
        max_batch_size (int): The maximum size of batches that are be fed to the LSTMCell. Each
            state variable in the result tuple will have max_batch_size rows in the first dimension.
    Returns:
        tuple[tf.contrib.rnn.LSTMStateTuple]: A tuple of LSTMStateTuples. Each of those will contain
            a zero cell and zero hidden state, each of shape [max_batch_size, state_size]. The
            length of the result tuple will be determined by the cell's number of layers (e.g. for
            an MultiRNNCell).
    """

    # For each layer, get the initial state tuple and make two variables out of it to enable setting
    # its value.
    state_variables = []
    for state_c, state_h in cell.zero_state(batch_size, tf.float32):
        state_variables.append(tf.contrib.rnn.LSTMStateTuple(
            tf.Variable(state_c, trainable=False),
            tf.Variable(state_h, trainable=False)))
    return tuple(state_variables)

In [4]:
def get_state_update_op(state_variables, new_states):
    """Returns an operation to update an LSTM's state variables.
    See get_state_variables() for more info.
    Args:
        state_variables (tuple[tf.contrib.rnn.LSTMStateTuple]): The LSTM's state variables.
        new_states (tuple[tf.contrib.rnn.LSTMStateTuple]): The new values for the state variables.
            new_states may have state tuples with state sizes < max_batch_size. Then, only the first
            rows of the corresponding state variables will be updated.
    Returns:
        tf.Operation: An operation that updates the LSTM's.
    """

    # Add an operation to update the train states with the last state tensors.
    update_ops = []
    for state_variable, new_state in zip(state_variables, new_states):
        # new_state[0] might be smaller than state_variable[0], because state_variable[0]
        # contains max_batch_size entries.

        # Get the update indices for both states in the tuple
        update_indices = (tf.range(0, tf.shape(new_state[0])[0]),
                          tf.range(0, tf.shape(new_state[1])[0]))
        update_ops.extend([
            tf.scatter_update(state_variable[0], update_indices[0], new_state[0]),
            tf.scatter_update(state_variable[1], update_indices[1], new_state[1])
        ])
    return tf.tuple(update_ops)

In [5]:
def get_state_reset_op(state_variables, cell, batch_size):
    """Returns an operation to set each variable in a list of LSTMStateTuples to zero.
    See get_state_variables() for more info.
    Args:
        state_variables (tuple[tf.contrib.rnn.LSTMStateTuple]): The LSTM's state variables.
        cell (tf.contrib.rnn.MuliRNNCell): An MultiRNNCell consisting of multiple LSTMCells.
        max_batch_size (int): The maximum size of batches that are be fed to the LSTMCell.
    Returns:
        tf.Operation: An operation that sets the LSTM's state to zero.
    """
    zero_states = cell.zero_state(batch_size, tf.float32)
    return get_state_update_op(state_variables, zero_states)

In [6]:
def get_num_params(num_inputs, num_layers, num_neurons, num_classes):
    """Returns the number of trainable parameters of an LSTM.
    Args:
        num_inputs (int): The input vector size
        num_layers (int): The number of layers in the LSTM
        num_neurons (int): The number of neurons / units per layer
        num_classes (int): The number of classes
    Returns:
        int: The number of trainable parameters
    """
    num_first_layer = 4 * (num_neurons * (num_inputs + num_neurons) + num_neurons)
    num_other_layer = 4 * (num_neurons * 2 * num_neurons + num_neurons)
    num_softmax = num_classes * num_neurons + num_classes

    return num_first_layer + (num_layers - 1) * num_other_layer + num_softmax

In [7]:
#Network parameters
start_epoch = 1
num_epochs = 10
num_layers = 2
num_neurons = 128
learning_rate = 0.0001
batch_size = 128
output_keep_var = 0.5
initial_learning_rate=0.001
max_batch_size=10000

In [8]:
X = tf.placeholder(tf.float32, [None, num_steps, num_inputs], name='input_placeholder')
y = tf.placeholder(tf.float32, [None, num_classes], name='labels_placeholder')
output_keep_prob = tf.placeholder_with_default(1.0, shape=(), name ="input_keep_prob")

def build_lstm_cell(num_neurons, output_keep_prob=1.0):
    """Returns a dropout-wrapped LSTM-cell.
    See https://stackoverflow.com/a/44882273/2628369 for why this local function is necessary.
    Returns:
        tf.contrib.rnn.DropoutWrapper: The dropout-wrapped LSTM cell.
     """
    lstm_cell = tf.contrib.rnn.LSTMCell(num_units=num_neurons, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_drop = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=output_keep_prob)
    return lstm_cell_drop

with tf.name_scope("LSTM"):
        with tf.name_scope("Cell"):
            multi_layer_cell = tf.contrib.rnn.MultiRNNCell([build_lstm_cell(num_neurons, output_keep_prob=output_keep_prob) for _ in range(num_layers)], state_is_tuple=True)
        with tf.variable_scope("LSTM_init_state"):
            states = get_state_variables(batch_size, multi_layer_cell)
        with tf.name_scope("Model"):
            # Unroll the LSTM
            outputs, new_states = tf.nn.dynamic_rnn(multi_layer_cell, X, initial_state=states, time_major = False, dtype=tf.float32)#[Batch_size, time_steps, num_neurons]
        # Add an operation to update the train states with the last state tensors.
        update_op = get_state_update_op(states, new_states)
        reset_state_op = get_state_reset_op(states, multi_layer_cell, max_batch_size)
        #Softmax_w is 2D, but the outputs are 3D (batch_size, num_timesteps, num_neurons),
        outputs = tf.transpose(outputs, [1, 0, 2]) # [num_timesteps, batch_size, num_neurons]
        outputs = tf.gather(outputs, int(outputs.get_shape()[0]) - 1) #[num_timesteps, Batch_size, num_neurons]

        with tf.variable_scope('softmax'):
            softmax_w = tf.get_variable("softmax_w", [num_neurons, num_classes], initializer=tf.truncated_normal_initializer(stddev=0.1))#[num_neurons, num_classes]
            softmax_b = tf.get_variable("softmax_b", [num_classes], initializer=tf.constant_initializer(0.1)) #[num_classes,]
            logits= tf.matmul(outputs, softmax_w) + softmax_b #[Batch_size, num_classes]
        with tf.name_scope('Predictions'):
            predictions = tf.nn.softmax(logits, name="predictions")  #[Batch_size, num_classes]
        with tf.name_scope('Accuracy'):
            correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        with tf.name_scope('Loss'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))
        with tf.name_scope('Train'):
            optimizer= tf.train.AdamOptimizer(learning_rate=learning_rate)
            trainer=optimizer.minimize(loss)

In [12]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    batchNum = mnist.train.num_examples // batch_size
    for batch in range(batchNum):
        xBatch, yBatch = mnist.train.next_batch(batch_size)
        xBatch = xBatch.reshape((batch_size, num_steps, num_inputs))
        outputs_train, update_op_train = sess.run([outputs, update_op], feed_dict={X: xBatch, y: yBatch, output_keep_prob:output_keep_var})